In [197]:
import MetaTrader5 as mt5
from datetime import datetime
import pandas as pd
import time 
import numpy as np

In [198]:
# String inicial
base_string = 'PETR'

# Gerar a lista com as variações de 'VALEA' até 'VALEZ'
result = [base_string + chr(i) for i in range(ord('A'), ord('Z') + 1)]

# Exibir o resultado
print(result[23])

PETRX


In [199]:
mt5.initialize()

mt5.login('LOGIN','SENHA', 'Servidor')
lista_opcoes = []


for nome in result:
    ticker = mt5.symbols_get(nome)
    # print(len(ticker))
    for j in range(1,len(ticker)):
        lista_opcoes.append(ticker[j].name)

lista_opcoes[5]

'PETRA14'

In [200]:
for i in range(len(lista_opcoes)):
    ticker = lista_opcoes[i]
    
    # Obter as informações do símbolo de uma vez
    mt5.symbol_select(ticker,True)

time.sleep(10)
    

In [201]:
dicionario_opcoes = {}
for i in range(len(lista_opcoes)):
    ticker = lista_opcoes[i]
    
    # Obter as informações do símbolo de uma vez
    
    symbol_info = mt5.symbol_info(ticker)
    ativo_base = mt5.symbol_info(ticker).basis
    mt5.symbol_select(ativo_base,True)
    if (ativo_base == ''):
        cota = 0
    else:
        cota = mt5.symbol_info(ativo_base).last


    if symbol_info is not None:
        # Construir o valor do dicionário com as informações
        dicionario_opcoes[ticker] = {
            'Last': symbol_info.last,
            'Bid': symbol_info.bid,
            'Ask': symbol_info.ask,
            'Volume': symbol_info.session_volume,
            'Vencimento': datetime.fromtimestamp(symbol_info.expiration_time).strftime('%Y-%m-%d'),
            'Strike': symbol_info.option_strike,
            'Cotacao_Ativo' : cota,
            # 'Vencimento': symbol_info.expiration_time,
            'Abertura': symbol_info.session_open
        }

# Exibir o dicionário para conferir
print(dicionario_opcoes)

{'PETRA109': {'Last': 24.64, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Vencimento': '2025-01-17', 'Strike': 10.94, 'Cotacao_Ativo': 38.9, 'Abertura': 0.0}, 'PETRA114': {'Last': 27.8, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 1600.0, 'Vencimento': '2025-01-17', 'Strike': 11.44, 'Cotacao_Ativo': 38.9, 'Abertura': 27.8}, 'PETRA119': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Vencimento': '2025-01-17', 'Strike': 1.19, 'Cotacao_Ativo': 38.9, 'Abertura': 0.0}, 'PETRA124': {'Last': 23.86, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Vencimento': '2025-01-17', 'Strike': 12.44, 'Cotacao_Ativo': 38.9, 'Abertura': 0.0}, 'PETRA139': {'Last': 24.3, 'Bid': 17.06, 'Ask': 0.0, 'Volume': 11000.0, 'Vencimento': '2025-01-17', 'Strike': 12.94, 'Cotacao_Ativo': 38.9, 'Abertura': 24.21}, 'PETRA14': {'Last': 29.75, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 100.0, 'Vencimento': '2025-01-17', 'Strike': 9.44, 'Cotacao_Ativo': 38.9, 'Abertura': 29.75}, 'PETRA142': {'Last': 31.13, 'Bid': 21.8, 'Ask': 0.0, 'Volume': 21900.0, 

In [202]:

grade_opcoes = pd.DataFrame(dicionario_opcoes).T

opcoes_negociadas = grade_opcoes[grade_opcoes['Volume'] != 0]

In [207]:
opcoes_negociadas

,Last,Bid,Ask,Volume,Vencimento,Strike,Cotacao_Ativo,Abertura
PETRA114,27.8,0.0,0.0,1600.0,2025-01-17,11.44,38.9,27.8
PETRA139,24.3,17.06,0.0,11000.0,2025-01-17,12.94,38.9,24.21
PETRA14,29.75,0.0,0.0,100.0,2025-01-17,9.44,38.9,29.75
PETRA142,31.13,21.8,0.0,21900.0,2025-01-17,8.19,38.9,31.4
PETRA16,22.88,0.0,0.0,1000.0,2025-01-17,16.44,38.9,22.88
...,...,...,...,...,...,...,...,...
PETRX572,14.0,0.0,0.0,6000.0,2024-12-20,52.44,38.9,14.0
PETRX584,17.3,0.0,0.0,2400.0,2024-12-20,57.44,38.9,17.3
PETRX593,0.06,0.05,0.06,15200.0,2024-12-20,33.44,38.9,0.05
PETRX620,0.01,0.0,0.02,600.0,2024-12-20,25.44,38.9,0.01


In [208]:
# LIMPA A MARKWT WATCH DO MT5

lista_remocao = list(grade_opcoes.index)

for nome in lista_remocao:
    mt5.symbol_select(nome,False)

mt5.symbol_select(ativo_base,False)

    

True

In [209]:

mt5.shutdown()


True